In [1]:
from src.simgraph import calc_simgraph
import src.spectral_utils as su
import src.graph_utils as gu
import src.localization as localization

import matplotlib.pyplot as plt

In [2]:
#For some setups its neccessary to allow tensorflow to allocate gpu memory
import tensorflow as tf
#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
#tf.config.experimental.set_memory_growth(physical_devices[0], True)


In [3]:
f1 = './testimages/splicing-01.TIF' #spliced image
f2 = './testimages/normal-01.TIF' #authentic image

fweights = './models/cam_256/-30' #path to model CNN weights
patch_size = 256 #patch size, must match associated weights file
overlap = 128 #patch sampling overlap

In [4]:
#compute forensic similarity graphs for both images
#see example 0 for what they look like
I1 = plt.imread(f1)[:,:,:3]
sg1 = calc_simgraph(image = I1,
                    f_weights_restore = fweights, patch_size = patch_size,
                    overlap = overlap)

I2 = plt.imread(f2)[:,:,:3]
sg2 = calc_simgraph(image = I2,
                    f_weights_restore = fweights, patch_size = patch_size,
                    overlap = overlap)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).



Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./models/cam_256/-30


100%|██████████| 165/165 [00:01<00:00, 85.10it/s]


INFO:tensorflow:Restoring parameters from ./models/cam_256/-30


100%|██████████| 165/165 [00:02<00:00, 81.80it/s]


In [5]:
## Forgery Detection - spliced image
#Low spectral gap (<100, or <0.5 normalized) values are indication of tampering
#high modularity values (>0.025) are indication of tampering

M = gu.sym_mat(sg1.mat) #symmetric similarity matrix for spliced image

L = su.laplacian(M) #laplacian matrix
gap = su.eigap01(L) #spectral gap
print(f'Spliced: Spectral Gap = {gap:.2f}')

normL = su.laplacian(M, laplacian_type='sym') #normalized laplacian matrix
normgap = su.eigap01(normL) #normalized spectral gap
print(f'Spliced: Normalized Spectral Gap = {normgap:.4f}')

g = gu.adj_to_graph(M) #convert to igraph Graph object
_, _, mod = gu.cluster_fastgreedy(g,weighted=True) #compute modularity
print(f'Spliced: Modularity Value = {mod:.4f}')


Spliced: Spectral Gap = 10.49
Spliced: Normalized Spectral Gap = 0.4321
Spliced: Modularity Value = 0.0293


In [6]:
## Forgery Detection - authentic image
#Low spectral gap (<100, or <0.5 normalized) values are indication of tampering
#high modularity values (>0.025) are indication of tampering

M = gu.sym_mat(sg2.mat) #symmetric similarity matrix for spliced image

L = su.laplacian(M) #laplacian matrix
gap = su.eigap01(L) #spectral gap
print(f'Authentic: Spectral Gap = {gap:.2f}')

normL = su.laplacian(M, laplacian_type='sym') #normalized laplacian matrix
normgap = su.eigap01(normL) #normalized spectral gap
print(f'Authentic: Normalized Spectral Gap = {normgap:.4f}')

g = gu.adj_to_graph(M) #convert to igraph Graph object
_, _, mod = gu.cluster_fastgreedy(g,weighted=True) #compute modularity
print(f'Authentic: Modularity Value = {mod:.4f}')

Authentic: Spectral Gap = 161.89
Authentic: Normalized Spectral Gap = 1.0027
Authentic: Modularity Value = 0.0060
